**tutorial here: https://www.sc-best-practices.org/cellular_structure/annotation.html#environment-setup**


**Environment setup
We’ll filter out some deprecation and performance warnings that do not affect our code:**

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

from numba.core.errors import NumbaDeprecationWarning

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

In [ ]:
#Load the needed modules:

import urllib.request
from pathlib import Path

import celltypist
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
# import scarches as sca   # skip Annotation by mapping to a reference
import seaborn as sns
from celltypist import models
from scipy.sparse import csr_matrix

In [ ]:
#One more pandas warning to filter:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [ ]:
#Set figure parameters:
sc.set_figure_params(figsize=(5, 5))

In [ ]:
# Load your already-preprocessed & clustered AnnData:
adata = sc.read("/home/jiawu2/pub_scRNAseq_run/SRR23934263_clustered.h5ad")

In [ ]:
# Quick sanity check: what was saved in the clustered object?
print("layers:", list(adata.layers.keys()))
print("obs keys (first 25):", list(adata.obs.keys())[:25])
print("var keys (first 25):", list(adata.var.keys())[:25])
print("obsm keys:", list(adata.obsm.keys()))
print("varm keys:", list(adata.varm.keys()))
print("obsp keys:", list(adata.obsp.keys()))
print("uns has neighbors?:", "neighbors" in adata.uns)
if "neighbors" in adata.uns:
    print("neighbors params:", adata.uns["neighbors"].get("params", {}))


## Annotation workflow (reusing existing clustering/UMAP)

This notebook **reuses** the existing:
- `adata.obs['leiden' ...]` cluster labels
- `adata.obsm['X_umap']` embedding
- `adata.obsp['connectivities']` neighbor graph

We will **not** recompute PCA / neighbors / UMAP unless they are missing.

We will:
1. Do **manual marker-based annotation** (dotplots / UMAP overlays)
2. Do **cluster DE-based inspection** (`rank_genes_groups`)
3. Optionally run **automated annotation (CellTypist)** on a **copy** of the data (so the embedding stays unchanged)


In [ ]:
import scanpy as sc
import numpy as np

# Preferred clustering resolution(s), in priority order
preferred = ["leiden_res1", "leiden_res0_5", "leiden_res0_25", "leiden"]

cluster_key = next((k for k in preferred if k in adata.obs), None)
if cluster_key is None:
    raise ValueError("No Leiden clustering found in adata.obs")

print("Using cluster key:", cluster_key)

sc.pl.umap(adata, color=cluster_key, legend_loc="on data", frameon=False)


## Manual annotation

In [ ]:
##known-marker based approach

marker_genes = {
    "Monocyte/Macrophage": ["FCN1", "CD14", "TCF7L2", "LYZ", "S100A8", "CD68",
                            "FCGR3A", "LYN",
    ], ##FCGR3A(CD16), LYZ(lysozyme)
    
    "cDC1": ["CLEC9A", "CADM1"],
    "cDC2": ["CST3", "COTL1", "LYZ", "DMXL2", "CLEC10A", "FCER1A",  ],      
    "Erythro": ["MKI67", "HBA1", "HBB"],
    "Early Erythro": ["CDK6", "SYNGR1",  "HBM", "GYPA", ],  # Note HBM and GYPA are negative markers
    
    "NK": ["NCAM1", "FCGR3A", "CCL5", "GNLY", "NKG7", "CD247", "GRIK4", "FCER1G",
           "TYROBP", "KLRG1", "FCGR3A"
    ], ## NCAM1(CD56), FCGR3A(CD16)
    
   # "ILC": ["ID2", "PLCG2", "GNLY", "SYNE1"],
    "Lymph prog": [
        "VPREB1", "MME", "EBF1","SSBP2", "BACH2",
        "CD79B", "IGHM",  "PAX5",  "PRKCE", "DNTT", "IGLL1",
    ],
    
    "B": ["CD19", "MS4A1","CD79A", "PAX5", "IGHD", "FCRL1", "IGHM"], #MS4A1(CD20),
      # Note IGHD and IGHM are negative markers
    "Transition B": ["MME","CD38", "CD24", "ACSM3", "MSI2"],
    
    "Plasma cells": ["SDC1", "MZB1", "HSP90B1", "FNDC3B", "PRDM1", 
                     "IGKC", "JCHAIN"], #SDC1(CD138)
    
    #"Plasmablast": ["XBP1", "RF4", "PRDM1", "PAX5"],  # Note PAX5 is a negative marker
    "CD4+ T activated": ["CD4", "IL7R", "TRBC2", "ITGB1"],
    "CD4+ T naive": ["CD4", "IL7R", "TRBC2", "CCR7"],
    "CD8+ T": ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    "T activation": ["CD3D", "CD3E", "CD69", "CD38"],  # CD69 much better marker!
    "T naive": ["CD3D", "CD3E", "LEF1", "CCR7", "TCF7"],
    "pDC": ["MZB1", "GZMB", "IL3RA", "COBLL1", "TCF4", "CD303"],
    
    "G/M prog": ["MPO", "KCNQ5", "CSF3R"],
    "MK/E prog": ["ZNF385D","ITGA2B", "RYR3", "PLCB1", ],  # Note PLCB1 is a negative marker
    
    "AML/LSC": ["CD34", "CD33","KIT", "FLT3", "IL3RA", "SPINK2", "CD274", "IL1RAP", "HOPX", "SOX4", "CD200", "CLEC12A", "CD38", "CD47"],
     "Myeloid stem cell": ["ADA", "ANXA1", "AZU1", "CEBPA", "CITED2", 
                              "CLEC11A", "JUND", "MPO"],
     "Leukemic stem cell": ["CD244", "CD200", "CD38", "KIT", "CD34", "CD47", "CD96", "BST1", "CLEC12A",
                           "HAVCR2", "IL1RAP", "MPO", "CD33", "IL3RA", "SPL1"],
     "Neoplastic Mono": ["ADA2", "AGTRAP", "ANXA2", "AP1S2", "CD14", "CD300E", "CD68", 
                         "CD48", "CSF3R", "S100A10",],
    
    "Normal HSPC": ["CD34", "THY1", "GATA2", "TAL1", "PROM1", "NRIP1", "MECOM", "ENG"], #PROM1(CD133), THY1(CD90) can mark quiescent HSCs. ENG-ENDOGLIN (Min Guan's CART)
    
}

In [ ]:
## Subset to only the detected markers. loop through all cell types and keep only the genes that we find in our adata object as markers for that cell type.  

marker_genes_in_data = {}
for ct, markers in marker_genes.items():
    markers_found = []
    for marker in markers:
        if marker in adata.var.index:
            markers_found.append(marker)
    marker_genes_in_data[ct] = markers_found

## Manual annotation: marker expression by cluster
We inspect canonical markers across clusters using dotplots/matrixplots and UMAP overlays.


In [ ]:
# Flatten marker genes (keep only those present)
all_markers = sorted({g for genes in marker_genes_in_data.values() for g in genes})
print(f"Markers found in data: {len(all_markers)}")

# Dotplot by cluster
if len(all_markers) > 0:
    sc.pl.dotplot(
        adata,
        var_names=marker_genes_in_data,   # dict: celltype -> markers
        groupby=cluster_key,
        standard_scale="var",
        dendrogram=False,
    )
else:
    print("No marker genes found in adata.var_names. Check gene naming (e.g., ENSG vs symbols).")


In [ ]:
# UMAP overlays for a few key markers (edit this list as needed)
quick_markers = []
for ct, genes in marker_genes_in_data.items():
    if genes:
        quick_markers.append(genes[0])
quick_markers = [g for g in quick_markers[:30] if g in adata.var_names]

print("Quick marker list:", quick_markers)
if quick_markers:
    sc.pl.umap(adata, color=quick_markers, ncols=4, frameon=False)


## Manual annotation: cluster marker discovery (DE per cluster)
We compute cluster-enriched genes to help interpret clusters beyond a predefined marker table.


In [ ]:
# Ensure we are using a normalized/log expression matrix for DE visualization.
# (This does NOT change the existing UMAP/Leiden geometry.)
preferred_layers = ["log1p_norm", "log1p", "scran_normalization"]
chosen = None
for lname in preferred_layers:
    if lname in adata.layers:
        chosen = lname
        break
if chosen is not None:
    adata.X = adata.layers[chosen]
    print(f"Using adata.layers['{chosen}'] as adata.X for DE/marker visualization.")
else:
    print("No recognized normalized layer found; leaving adata.X as-is.")

# Rank genes by cluster (uses adata.X)
sc.tl.rank_genes_groups(
    adata,
    groupby=cluster_key,
    method="wilcoxon",
    pts=True
)
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False)


In [ ]:
# Optional: show a heatmap of top cluster markers
sc.pl.rank_genes_groups_heatmap(adata, n_genes=10, groupby=cluster_key, show_gene_labels=True)


In [ ]:
##show expression of the markers using the calculated UMAP

Myeloid_cts = [
    "Monocyte/Macrophage",
    "AML/LSC",
    "Leukemic stem cell",
    "Neoplastic Mono",
    "G/M prog",
    "pDC",
    "Normal HSPC",
    ]



In [ ]:
for ct in Myeloid_cts:
    if ct not in marker_genes_in_data:
        print(f"Skipping {ct}: not found in marker_genes_in_data")
        continue

    genes = [g for g in marker_genes_in_data[ct] if g in adata.var_names]
    if len(genes) == 0:
        print(f"Skipping {ct}: no markers found in dataset")
        continue

    print(f"{ct.upper()}:")
    sc.pl.umap(
        adata,
        color=genes,
        vmin=0,
        vmax="p99",        # robust scaling to avoid outliers
        sort_order=False, # avoid visual bias
        frameon=False,
        cmap="Reds",
    )
    print("\n\n")

In [ ]:
##show expression of the markers using the calculated UMAP

Other_cell_cts = [
   "Erythro", "Early Erythro", "NK", "Lymph prog", "B", "Transition B", "Plasma cells",  
    "T activation", "T naive", "pDC", "CD4+ T naive", "CD8+ T", "MK/E prog",
]

In [ ]:
for ct in Other_cell_cts:
    if ct not in marker_genes_in_data:
        print(f"Skipping {ct}: not found in marker_genes_in_data")
        continue

    genes = [g for g in marker_genes_in_data[ct] if g in adata.var_names]
    if len(genes) == 0:
        print(f"Skipping {ct}: no markers found in dataset")
        continue

    print(f"{ct.upper()}:")
    sc.pl.umap(
        adata,
        color=genes,
        vmin=0,
        vmax="p99",        # robust scaling to avoid outliers
        sort_order=False, # avoid visual bias
        frameon=False,
        cmap="Reds",
    )
    print("\n\n")

In [ ]:
# visualize this using a dotplot:

Myeloid_markers = {
    ct: [m for m in ct_markers if m in adata.var.index]
    for ct, ct_markers in marker_genes.items()
    if ct in Myeloid_cts
}

In [ ]:
sc.pl.dotplot(
    adata,
    groupby="leiden_res1",
    var_names=Myeloid_markers,
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
)

In [ ]:
# visualize this using a dotplot:

Other_cell_markers = {
    ct: [m for m in ct_markers if m in adata.var.index]
    for ct, ct_markers in marker_genes.items()
    if ct in Other_cell_cts
}

sc.pl.dotplot(
    adata,
    groupby="leiden_res1",
    var_names=Other_cell_markers,
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
)

## Assign human-readable cell type labels
Create a mapping from Leiden cluster → cell type label after inspecting markers/DE.


In [ ]:
cl_annotation = {
    "6": "AML (CD34+ Spink2low IL1RAPhi CD33low CLL1low CD47hi)",
    "1": "AML (CD34+ Spink2low IL1RAPhi CD33low CLL1low CD47hi)",

    "5": "Mono/Macro (CD68+ FCN1+ LYZ+ S100AB+ CLL1+ CD123+)",

    "9": "AML (CD34+ CD200+ FLT3+ KIThi)",

    "3": "AML (CD34+ CD38low CD33hi IL1RAPlow CD47low)",

    "12": "AML? (CD34low CD38low KIT+ MPO+ TIM3+ CD123+)",

    "11": "G/M prog (MPO+ CSF3R+)",

    "7": "norm HSPC (CD34+ GATA2+ NR1P1+)",

    "4": "Erytho (MKI67+ HBA1+ HBB+ GYPA+ TAL1+)",
    "15": "Erytho (MKI67+ HBA1+ HBB+ GYPA+ TAL1+)",

    "14": "CD8 T (CD8A+ CCL5+ CD3D+ CD69+ TCF7+)"
}


In [ ]:
adata.obs["manual_celltype_annotation"] = (
    adata.obs["leiden_res1"]
    .astype(str)
    .map(cl_annotation)
)


In [ ]:

sc.pl.umap(
    adata,
    color=["leiden_res1", "manual_celltype_annotation"],
    ncols=2,
    legend_loc="right margin",
    frameon=False
)


In [ ]:
print(adata.obs["manual_celltype_annotation"].value_counts(dropna=False))


In [ ]:
adata.write("SRR23934263_manual_annotated.h5ad")
